In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
from fbprophet import Prophet
model = Prophet()

In [3]:
!pip install Prophet

     |████████████████████████████████| 65 kB 303 kB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 49 kB 944 kB/s            
  Created wheel for Prophet: filename=prophet-1.0.1-py3-none-any.whl size=9432067 sha256=750cf12cdbf3dd2f87eeef1de78f7221970f4988160c6e7aa766c3ae134c86f2
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built Prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


In [4]:
from pandas import read_csv
path = '/kaggle/input/store-sales-time-series-forecasting/train.csv'
df = read_csv(path, header=0)
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
from pandas import to_datetime
# prepare expected column names
df.columns = ['id','ds','store_nbr','family', 'y','onpromotion']
df['ds']= to_datetime(df['ds'])
# define the model
model = Prophet()
# fit the model
m = model.fit(df)

Initial log joint probability = -156.737
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 136)

Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 136)

Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 136)

Error evaluating model log probability: Non-finite gradient.

      99   1.27077e+07   3.38615e-05       38790.4           1           1      136   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199   1.27081e+07   8.59518e-05       74153.2           1           1      248   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     271   1.27082e+07   1.29496e-05       21426.8     0.07606      0.9747      330   
Optimization terminated normally: 
  Convergence detec

In [6]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2044,2018-08-11
2045,2018-08-12
2046,2018-08-13
2047,2018-08-14
2048,2018-08-15


In [7]:
# Python
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2044,2018-08-11,620.807202,-926.228446,2286.772335
2045,2018-08-12,651.080988,-768.113522,2105.349532
2046,2018-08-13,533.702627,-929.506452,2029.286719
2047,2018-08-14,507.889313,-994.638767,2058.363652
2048,2018-08-15,522.257943,-968.415150,2137.479393
